In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

**References**
- [SELF-Instruct paper](https://arxiv.org/pdf/2212.10560.pdf)
- [Self-Alignment with Instruction Backtranslation](https://arxiv.org/pdf/2308.06259.pdf)
- [Large Language Models can Self-Improve](https://arxiv.org/pdf/2210.11610.pdf)

# Using an LLM to generate Instruction Following examples

In the module on [Instruction Following](LLM_Instruction_Following.ipynb)
- we motivated the use of Fine-Tuning a LLM
- to exhibit Instruction Following behavior

Recall: an example of Instruction Following behavior is a triple

$$\langle \text{Instruction}, \text{Context}, \text{Response} \rangle $$

for example
- Instruction: "Tell me the word that is the opposite of the word that I input"
- Context: "Input: Stop"
- Response: "Go"

The Instruction describes the task to be accomplished
- relationship between Input and Response
- the Input/Response pair is an exemplar for this task

In this module, we explore methods
- to generate these fine-tuning examples
- to improve examples

# Using an LLM to generate Instruction Following examples

**Reference**

[SELF-Instruct paper](https://arxiv.org/pdf/2212.10560.pdf)

Is there an alternative to the labor-intensity of constructing Instruction Following examples by human ?

These examples are pairs of an Instruction part, and a Target Output part.

The idea of the [SELF-Instruct paper](https://arxiv.org/pdf/2212.10560.pdf)
is to create Synthetic examples
- using [In-Context Learning](In_Context_Learning.ipynb)

We can imagine the process as
- starting with a small number $k$ of human-constructed examples

$$
\begin{array}[lll] \\
\langle \text{Instruction}^{(1)}, \text{Context}^{(1)}, \text{Response}^{(1)} \rangle \\
\vdots \\
\langle \text{Instruction}^{(k)}, \text{Context}^{(k)}, \text{Response}^{(k)} \rangle \\
\end{array}
$$

- which are used as exemplars
- in a *few-shot* learning prompt
- that an LLM completes into a new Instruction Following example

$$
\langle \text{Instruction}^{(k+1)}, \text{Context}^{(k+1)}, \text{Response}^{(k+1)} \rangle \\
$$

The human and LLM generated examples can then be used to Fine Tune an LLM to better demonstrate Instruction Following

In actuality
- the Instruction part of the triple 

$$
\langle \text{Instruction}^{(k+1)}, \text{Context}^{(k+1)}, \text{Response}^{(k+1)} \rangle \\
$$

- is generated first
- using In-Context learning

Given the newly generated instruction $$\text{Instruction}^{(k+1)}$$
- In-Context learning is used again
- to generate the associated Context and Response $$\text{Context}^{(k+1)}, \text{Response}^{(k+1)} $$



The process for generating each part of the triple 
- is illustrated by a diagram
- with the steps described in detail below
<br>
<img src="images/selfinstruct_process.png">

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=2

## Generating the Instruction part of an Instruction-Output example

The first step is to generate the  first Instruction part (i.e., the Instruction) of the triple

$$
\langle \textbf{Instruction}^{(k+1)}, \text{Context}^{(k+1)}, \text{Response}^{(k+1)} \rangle \\
$$

using $k$ exemplars
$$
\begin{array}[lll] \\
\langle \text{Instruction}^{(1)} \rangle \\
\vdots \\
\langle \text{Instruction}^{(k)} \rangle \\
\end{array}
$$

**See the box labeled "Step 1" in the illustration above**

The precise template for the exemplars is show below.




<img src="images/selfinstruct_task_generation_prompts.png" width=90%>

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=15

With the above template, we expect the LLM
- to generate a continuation of the prompt 
    - ending with `Task 9: `
- which is the Instruction part of a new task

## Generating the Context/Respones (Input/Output) part, given an Instruction

Once we have generated a the Instruction part
$$\text{Instruction}^{(k+1)}$$

of the new synthetic example, we need to generate the Context and Response
- using exemplars

$$
\begin{array}[lll] \\
\langle \text{Instruction}^{(1)}, \text{Context}^{(1)}, \text{Response}^{(1)} \rangle \\
\vdots \\
\langle \text{Instruction}^{(k)}, \text{Context}^{(k)}, \text{Response}^{(k)} \rangle \\
\end{array}
$$

and prompt
$$
\text{Instruction}^{(k+1)}
$$

with the expectation that the LLM's continuation will be
$$
\text{Context}^{(k+1)}, \text{Response}^{(k+1)}
$$


**See the box labeled "Step 3" in the diagram above**

For Classification tasks, the prompt might look like this

    Task: Classify the sentiment of the sentence into positive, negative, or mixed
    
    Example 1
    Sentence: I enjoy the flavor of the restaurant but their service is too slow.
    Class Label: mixed
    
    Example 2
    Sentence: I had a great day today. The weather was beautiful and I spent time with friends.
    Class label: Positive
    
    
    Task: Tell me if the following email is a promotion email or not.
    
    Email: Check out our amazing new sale! We’ve got discounts on all of your favorite products.
    Class label: Promotion

    Email: We hope you are doing well. Let us know if you need any help.
    Class label: Not Promotion
    
    Task: {instruction for the target task}

The last line above contains a place holder for the Instruction of the Target Task

$$
\text{Instruction}^{(k+1)}
$$

that we created in Step 1.

Here is an example of the template from the paper

<img src="images/selfinstruct_generated_instances.png">

Attribution: https://arxiv.org/pdf/2212.10560.pdf#page=16

### Difficulties in Generating the Input/Output part: Classification tasks

Although the few-shot learning approach to generating an Input/Output given an Instruction 
- seems straightforward
- the authors encountered difficulties when generating Input/Output for Classification tasks

Consider the an Instruction Following example for a Classification task

    Task: Classify the sentiment of the sentence into positive, negative, or mixed
    
    Example 1
    Sentence: I enjoy the flavor of the restaurant but their service is too slow.
    Class Label: mixed
 

The authors found that the response generated by the LLM (e.g., Classification examples)
- were examples whose Class Label's 
- were *not well-distributed* among all possible labels 
    - examples with certain labels were either over or under represented

This was attributed to the *format* of the example called *Input-first* format
$$
\langle \text{Instruction}^{(i)}, \text{Context}^{(i)}, \text{Response}^{(i)} \rangle
$$

where 
- $\text{Context}^{(i)}$ is the `Additional Input`
- $\text{Response}^{(i)}$ is the `Class label`



The solution was to change the example format to *output-first*
$$
\langle \text{Instruction}^{(i)},  \text{Response}^{(i)}, \text{Context}^{(i)} \rangle
$$

where the Response (`Class label`) **precedes** the Context (`Additional Input`)

For example:

     Task: Classify the sentiment of the sentence into positive, negative, or mixed

     Example 1
        Class Label: mixed
        Sentence: I enjoy the flavor of the restaurant but their service is too slow.
        

        Example 2
        Class label: Positive
        Sentence: I had a great day today. The weather was beautiful and I spent time with friends.
        


This is an example of Prompt Engineering
- In-context learning seems very sensitive to the format of prompts
- There is a skill of engineering a prompt to elicit the desired behavior

This feels similar to the idea behind Chain of Thought prompting
- by presenting `Class Label` first
- the model seems better conditioned to generate a less biased distribution of labels

# Generating Instructions via  Backtranslation

We now present an alternate method for generating the Instruction part of 
$$
\langle \text{Instruction} \rangle
$$
of an Instruction Following example
$$
\langle \text{Instruction}, \text{Context}, \text{Response} \rangle
$$


The idea seems, at first, to be backwards:
- Generate an instance of Instruction Following behavior
$$\langle\x, \y \rangle =  \langle \text{Instruction}, \text{Response} \rangle$$
- by starting with a
 $\text{Response}$
- and using an LLM to create the $\text{Instruction}$

Essentially
- we start with *un-labeled data* (`Response`)
- and create a *label* (`Instruction`)

The advantage of this approach is that
- un-labeled data is plentiful
    - almost any block of text
- but labeled data ( `Response/Instruction` pairs) is scarce.

So, starting with a plentiful resource, we create the scarce resource
- i.e, Instruction Following example triplet

The method is called *Back Translation*.

- given a *small* "seed"  of Instruction/Response pairs
$$\langle\x, \y \rangle =  \langle \text{Instruction}, \text{Response} \rangle$$
- create an inverse dataset of Response/Instruction pairs by reversing the features and targets
$$\langle\y, \x \rangle =  \langle  \text{Response}, \text{Instruction} \rangle$$

We use the inverse dataset to train a model $M_{yz}$
- given a Response ($\y$)
- to create an Instruction ($\z = \hat\x$)

by fine-tuning an LLM to predict $\text{Instruction}$ from $\text{Response}$


 $M_{yz}$
 - when fed by set of Responses $\{ y_i \}$
     - the Unlabeled Data in the diagram
 - creates new instances of Instruction/Response examples
 
This results in an enlarged set of Instruction/Response examples
- the original human generated examples
- augmented by the Synthetic examples creates by  $M_{yz}$

in a process called *Self Augmentation*


With the newly extended set of seed Instruction/Response pairs
- we have more exemplars
- which we can use as a seed to another iteration of  $M_{yz}$
    - the enlarged set of exemplars may result in *better* synthetic Instruction/Response pairs

We can iterate on this process multiple times
- using the Augmented set of Instruction/Response pairs from step $i$
- as the "seed" for iteration $(i +1)$ of the process

Here is the workflow:

<table>
    <center><strong>Instruction Backtranslation</strong></center>
    <tr>
        <img src="images/instruction_backtranslation.png" width=70%>
    </tr>
    
Attribution: https://arxiv.org/pdf/2308.06259.pdf#page=2
</table>

## Selecting the best synthetic examples for augmentation

The quality of the synthetic examples created at each step may not be uniformly high.

It would be desirable 
- to select only the best examples to use
- in augmenting the seed examples of each iterative Step.

How can we rate the quality of a synthetic example ?

Ask the LLM to do it for you ! 

Using just the seed data
- fine tune a "first generation" LLM
    - denoted $M_0$
- to create a quality score of examples

The following prompt requests that the LLM evaluate the
synthetic example using a rating scale of $1$ (low quality) to $5$ (high quality)

<table>
    <center><strong>Instruction Backtranslation Curation</strong></center>
    <tr>
        <img src="images/instruction_backtranslation_curating.png" width=70%>
    </tr>
    
Attribution: https://arxiv.org/pdf/2308.06259.pdf#page=4
</table>

Use $M_0$ to
- select the best first generation augmented examples (from the first iteration)

The next generation augmented data set is
- the prior generation 
- augmented with the best (highest quality scores) of the new generation

Now that we have
- an augmented (high quality) "generation $i$" set of seed examples

we continue our iterative process
- creating a more powerful scoring LLM $M_i$
- to create an augmented high quality "generation $i+1$" set of examples

This is an example of [LLM Self-Improvement](LLM_Self_Improvement.ipynb).

In [2]:
print("Done")

Done
